In [ ]:
import os 
os.chdir("..")

In [ ]:
import polars as pl
import geopandas as gpd
import pandas as pd
import arviz as az
import numpy as np
import bambi as bmb
from src.data_pull import DataPull
import requests
import missingno as msno


az.style.use("arviz-darkgrid")


dp = DataPull(database_file="data3.ddb")

In [ ]:
df_min = dp.pull_min_wage()
df_min = df_min.with_columns(min_wage=pl.col("min_wage").str.replace("$","",literal=True))
df_shpae = pl.from_pandas(dp.pull_states_shapes().drop("geometry", axis=1))
df_min = df_min.join(
    df_shpae,
    on="state_name",
    how="inner",
    validate="m:1"
)
df_min = df_min.with_columns(pl.col("year").cast(pl.String))
var = "area_fips,year,qtr,industry_code,agglvl_code,month1_emplvl,month2_emplvl,month3_emplvl,total_qtrly_wages,avg_wkly_wage,qtrly_estabs"
df = dp.conn.sql(
    f"""
    SELECT {var} FROM 'QCEWTable' 
        WHERE agglvl_code=74;
    """
).pl()
df = df.with_columns(
    area_fips=pl.col("area_fips").str.zfill(5)
)
df = df.with_columns(
    fips=pl.col("area_fips").str.slice(0,2)
)
df = df.join(
    df_min,
    on=["fips","year"],
    how="inner",
    validate="m:1"
)

In [ ]:
naics_code = [
        "11",
        "21",
        "22",
        "31-33",
        "42",
        "48-49",
        "51",
        "55",
        "61",
        "71",
        "72",
        "81",
        "92",
        "23",
        "44-45",
        "52",
        "54",
        "56",
        "62",
    ]

In [ ]:
fips_list = dp.pull_states_shapes()["fips"].to_list()
# not_valid =["17","01"]
# 
not_valid = ['17',
 '27',
 '50',
 '31',
 '22',
 '13',
 '01',
 '39',
 '45',
 '40',
 '47',
 '32',
 '26',
 '05',
 '28',
 '30',
 '18',
 '72',
 '51']
fips_list = list(set(fips_list) - set(not_valid))

for fips in fips_list:
    for naics in naics_code:
        data = df.filter((pl.col("fips") == fips) & (pl.col("industry_code") == naics))
        data = data.with_columns(
    k_index=(pl.col("min_wage").cast(pl.Float64) * 8 * 5) / pl.col("avg_wkly_wage"),
    employment=(pl.col("month1_emplvl") + pl.col("month2_emplvl") + pl.col("month3_emplvl"))/3
)
        data = data.with_columns(
            log_k_index=pl.col("k_index").log(),
            log_employment=pl.col("employment").log()
        )
        data = data.to_pandas()
        data["date"] = data["year"].astype(int) * 10 + data["qtr"].astype(int)
        data["date"] = data["date"].astype("category")
        data["area_fips"] = data["area_fips"].astype("category")
        data = data.sort_values(["year", "qtr", "area_fips"]).reset_index(drop=True)
        data = data.replace([np.inf, -np.inf], np.nan)
        model = bmb.Model(
    "log_employment ~ 0 + area_fips + date + log_k_index",
    data,
    dropna=True,
)

        results = model.fit(
        cores=10,
        chains=10,
)
        az.to_netcdf(results, f"data/processed/results_{fips}_{naics}.nc")

In [ ]:
data.with_columns(min_wage=pl.col("min_wage").str.replace("$","", literal=True))

In [ ]:
data = df.filter((pl.col("fips") == "06") & (pl.col("industry_code") == "21"))
data = data.with_columns(
    k_index=(pl.col("min_wage").cast(pl.Float64) * 8 * 5) / pl.col("avg_wkly_wage"),
    employment=(pl.col("month1_emplvl") + pl.col("month2_emplvl") + pl.col("month3_emplvl"))/3
)
data = data.with_columns(
    log_k_index=pl.col("k_index").log(),
    log_employment=pl.col("employment").log()
)
data = data.to_pandas()
data["date"] = data["year"].astype(int) * 10 + data["qtr"].astype(int)
data["date"] = data["date"].astype("category")
data["area_fips"] = data["area_fips"].astype("category")
data = data.sort_values(["year", "qtr", "area_fips"]).reset_index(drop=True)
data = data.replace([np.inf, -np.inf], np.nan)
data

In [ ]:
model = bmb.Model(
    "log_employment ~ 0 + area_fips + date + log_k_index",
    data,
    dropna=True,
)

results = model.fit(
        sample_kwargs={"draws": 500, "tune": 500, "target_accept": 0.8},
        cores=10,
        chains=10,
)

In [ ]:
# Plot posteriors
az.plot_trace(
    results,
    compact=True,
)

In [ ]:
az.summary(results)

In [ ]:
df.join(
    df_min,
    on=["fips","year"],
    how="inner",
    validate="m:1"
)

In [ ]:
df

In [ ]:
var = "area_fips,year,qtr,industry_code,agglvl_code,month1_emplvl,month2_emplvl,month3_emplvl,total_qtrly_wages,avg_wkly_wage,qtrly_estabs"
df = dp.conn.sql(
    f"""
    SELECT {var} FROM 'QCEWTable' 
        WHERE agglvl_code=74;
    """
).pl()
df.filter(pl.col("avg_wkly_wage") == pl.col("avg_wkly_wage").max())

In [ ]:
pl.read_csv("out.csv")

In [ ]:
dp.conn.sql("SELECT * FROM 'QCEWTable';").write_csv("out.csv")

In [ ]:
temp = dp.conn.sql("SELECT DISTINCT area_fips FROM 'QCEWTable' WHERE year=2018 AND qtr=1;").df()
temp["area_fips"] = temp["area_fips"].str.zfill(5)
temp = temp["area_fips"].to_list()

In [ ]:
gdf = gpd.read_file("data/raw/tl_2024_us_county.zip")
gdf["county_id"] = gdf["STATEFP"] + gdf["COUNTYFP"]
remove_list_sates = ["66", "69", "60", "09", "15", "69", "02"]
remove_list_counties = ["46102"]
gdf = gdf[~gdf["STATEFP"].isin(remove_list_sates)]
gdf = gdf[~gdf["county_id"].isin(remove_list_counties)]
county_list = list(gdf["county_id"].values)

In [ ]:
gdf = gpd.read_file("data/raw/tl_2024_us_county.zip")
gdf = gdf.rename(columns={"GEOID":"geo_id", "NAME":"county_name","geometry":"geom"})
gdf[["geo_id","county_name", "geom"]]